<h1><center>Machine Learning-basierte Extraktion von phänotypischen Kennzahlen aus UAV-Bildern</center></h1>


<h3><center> harvest_classificator.ipynb: Bestimmung der Erntestatue  </center></h3>


Dieses Notebook enthält den Implementierungsprozess zur Bestimmung des Erntestatus, durch den die Pflanzen als geerntet oder nicht geerntet klassifiziert werden. Es beschreibt ausführlich die Schritte der Datenorganisation sowie die Trainingsprozesse für das Klassifikationsmodell.

## Bibliotheken

In [9]:
# Standard Libraries
import os
import glob as gb
import random

# External Libraries
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image, UnidentifiedImageError
from keras.preprocessing import image as kimage
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

## Datenvorbereitung

##### Harvested = 1 / Non-Harvested = 0

### Extraktion von Pflanzen-IDs

Diese Funktion wählt die plant_IDs der Pflanzen aus, die ein Bild im Datensatz haben. Dies wird durchgeführt, weil die Annotationsdatei viele plant_IDs enthält, von denen viele kein Bild haben. Diese plant_IDs werden in einem ersten Schritt extrahiert und in einem Dataframe organisiert. Dieses Dataframe wird verwendet, um Merkmale aus den ausgewählten Bildern zu extrahieren.

In [24]:
def create_dataframe(base_dir, sub_dirs):
    data = {'plant_id': [], 'harvested': []}
    
    for dir_name in sub_dirs:
        dir_path = os.path.join(base_dir, dir_name)
        
        for filename in os.listdir(dir_path):
            if filename.endswith('.jpg'):
                plant_id = os.path.splitext(filename)[0]  
                data['plant_id'].append(plant_id)
                data['harvested'].append(None) 

    df = pd.DataFrame(data)
    return df

base_dir = 'raid/GrowliFlowerO/Field2/'
sub_dirs = ["2021_06_16", "2021_07_07", "2021_07_29", "2021_08_19", "2021_08_30", "2021_06_23", "2021_07_12", "2021_08_04", "2021_08_23", "2021_09_03", "2021_07_01", "2021_07_20", "2021_08_11", "2021_08_25", "2021_09_08"]

field2_plants_df = create_dataframe(base_dir, sub_dirs)
#field2_plants_df.to_csv('raid/field2_plants.csv')

### Extraktion von Pflanzen, die als geerntet annotiert sind

In [66]:
image_paths = {}
df = pd.read_csv('raid/field1_plants.csv')
base_dir = 'raid/Annotations/Field1/'
for index, row in df.iterrows():
    plant_id = row['plant_id']
    parts = plant_id.split('_')
    date = '_'.join(parts[0:3])
    plot = parts[4][:5] 
    row = parts[5][0]
    position = parts[5][1:]
    annotation_path = os.path.join(base_dir, date, plot, f'{date}_{plot}.csv')
    dff = pd.read_csv(annotation_path)
    Column = 'Harvested?'
    value = dff[(dff['Row'] == row) & (dff['Position'] == int(position))][Column].values
    if value == 'x' or value == 'X':
       image_path = 'raid/GrowliFlowerO/Field1/' + date + '/' + plant_id + '.jpg'
       image_paths[date].append(image_path)

### Extraktion von Pflanzen, die als Nicht-geerntet annotiert sind

In [125]:
image_paths = {}
df = pd.read_csv('raid/field2_plants.csv')
base_dir = 'raid/Annotations/Field2/'
for index, row in df.iterrows():
    plant_id = row['plant_id']
    parts = plant_id.split('_')
    date = '_'.join(parts[0:3])
    plot = parts[4][:5] 
    row = parts[5][0]
    position = parts[5][1:]
    Column = 'Harvested?'
    annotation_path = os.path.join(base_dir, date, plot, f'{date}_{plot}.csv')
    dff = pd.read_csv(annotation_path)
    value = dff[(dff['Row'] == row) & (dff['Position'] == int(position))][Column].values
    image_path = 'raid/GrowliFlowerO/Field2/' + date + '/' + plant_id + '.jpg'
    if value != 'x' or value != 'X':
        if date not in image_paths:
            image_paths[date] = []
        image_paths[date].append(image_path)


Die Menge der Pflanzen, die als geerntet bezeichnet sind, übersteigt bei weitem die Menge der Pflanzen, die als nicht geerntet gekennzeichnet sind. Um einen ausgewogenen Datensatz zu erhalten, auf dem das Klassifizierungsmodell trainiert werden kann, wird nur ein Teil der geernteten Pflanzen im Datenframe gespeichert.

In [115]:
selected_paths = []

for date, paths in image_paths.items():
    if date in ['2020_08_12', '2020_08_19', '2020_08_25', '2020_09_02', '2020_09_08']:
        selected_paths.extend(random.sample(paths, min(50, len(paths))))
    elif date in ['2020_09_17', '2020_09_22', '2020_10_06', '2020_10_19', '2020_10_27']:
        selected_paths.extend(random.sample(paths, min(200, len(paths))))

In [138]:
for path in selected_paths:
    field2_non_harvested.loc[len(field2_non_harvested), 'plant_id'] = path

#### Kombination der Anmerkungen

In [148]:
annotations = pd.concat([harvested_plants, non_harvested_plants], ignore_index=True)
len(annotations)

3228

##### Dies ist der Trainingsdatensatz

In [10]:
annotations_path = 'raid/harvested_non_harvested_dataset.csv'
annotations = pd.read_csv(annotations_path)
annotations

,Unnamed: 0,plant_id,isHarvested
0,0,raid/GrowliFlowerO/Field1/2020_11_02/2020_11_0...,1
1,1,raid/GrowliFlowerO/Field1/2020_11_02/2020_11_0...,1
2,2,raid/GrowliFlowerO/Field1/2020_11_02/2020_11_0...,1
3,3,raid/GrowliFlowerO/Field1/2020_11_02/2020_11_0...,1
4,4,raid/GrowliFlowerO/Field1/2020_11_02/2020_11_0...,1
...,...,...,...
3223,3223,raid/GrowliFlowerO/Field2/2021_09_08/2021_09_0...,0
3224,3224,raid/GrowliFlowerO/Field2/2021_09_08/2021_09_0...,0
3225,3225,raid/GrowliFlowerO/Field2/2021_09_08/2021_09_0...,0
3226,3226,raid/GrowliFlowerO/Field2/2021_09_08/2021_09_0...,0


# Training des Klassifikationsmodell

## Merkmalsextraktion aus Bildern

In [21]:
image_paths = annotations['plant_id']
binary_targets = annotations['isHarvested']

### Extraktion mit VGG16

In [11]:
from keras.applications.vgg16 import VGG16, preprocess_input

def VGG16_feature_extratctor(image_path):
    model = VGG16(weights='imagenet', include_top=False)
    img = kimage.load_img(image_path, target_size=(490, 490))
    x = kimage.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    VGG16_features = model.predict(x)
    return VGG16_features

In [22]:
X_features = np.array([VGG16_feature_extratctor(path) for path in image_paths])
X_features = X_features.reshape(X_features.shape[0], -1)  # Flatten the features

2024-01-01 20:32:32.651641: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10096 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:73:00.0, compute capability: 7.5
2024-01-01 20:32:33.538957: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-01 20:32:33.640984: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 0s 102ms/step


KeyboardInterrupt: 

Die extrahierten Merkmale werden in einer Numpy-Datei für weitere Verwendungen gespeichert

In [ ]:
np.save('raid/harvest_detection_features.npy', X_features)

### Extraktion mit InceptionV3

In [6]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input

def InceptionV3_feature_extractor(image_path):
    model = InceptionV3(weights='imagenet', include_top=False)
    img = kimage.load_img(image_path, target_size=(299, 299))  # InceptionV3 erwartet eine Eingabe der Größe (299, 299)
    x = kimage.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    inception_features = model.predict(x)
    return inception_features

In [47]:
X_features = np.array([InceptionV3_feature_extractor(path) for path in image_paths])
X_features = X_features.reshape(X_features.shape[0], -1)  # Flatten the features

1/1 [==============================] - 1s 935ms/step


In [48]:
np.save('raid/harvest_detection_features2.npy', X_features)

### Extraktion mit Resnet50

In [30]:
from keras.applications.resnet50 import ResNet50, preprocess_input

def ResNet50_feature_extractor(image_path):
    model = ResNet50(weights='imagenet', include_top=False)
    img = kimage.load_img(image_path, target_size=(224, 224))  # ResNet50 erwartet eine Eingabe der Größe (224, 224)
    x = kimage.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    resnet_features = model.predict(x)
    return resnet_features

In [51]:
X_features = np.array([ResNet50_feature_extractor(path) for path in image_paths])
X_features = X_features.reshape(X_features.shape[0], -1)  # Flatten the features

1/1 [==============================] - 1s 1s/step


2024-01-02 00:15:29.074077: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 1s 724ms/step


In [52]:
np.save('raid/harvest_detection_features3', X_features)

### Kombinieren der extrahierten Merkmale

In [23]:
# Laden der von jedem Modell extrahierten Merkmale
VGG16_features = np.load('raid/harvest_detection_features.npy')
InceptionV3_features = np.load('raid/harvest_detection_features2.npy')
ResNet50_features = np.load('raid/harvest_detection_features3.npy')

In [9]:
# Verkettung der Merkmalen
X_features = np.concatenate([VGG16_features, InceptionV3_features, ResNet50_features], axis=1)

##### Die Idee war, mit mehreren Ansätzen Merkmale aus Bildern zu extrahieren und das Klassifizierungsmodell mit diesen zu trainieren. Diese Idee hat nicht die erwarteten Ergebnisse geliefert.
##### Tatsächlich hat sich gezeigt, dass das Training der Modelle mit Merkmalen, die nur mit der VGG16-Architektur extrahiert wurden, besser funktioniert und gute Ergebnisse liefert.

## Modelltraining

Um die höchste Genauigkeit zu gewährleisten, werden mehrere Klassifizierungsmodelle trainiert, von denen dasjenige mit dem besten Ergebnis ausgewählt wird.

### Training mit SVM

In [27]:
from sklearn.svm import SVC

# Aufteilen der Daten in Trainings- und Testsets
X_train, X_test, y_train, y_test = train_test_split(VGG16_features, binary_targets, test_size=0.2, random_state=42)

# Initialisieren des SVM-Modells
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')  # Dies sind Standardparameter, die möglicherweise angepasst werden müssen

# Trainieren des Modells mit den Trainingsdaten
svm_model.fit(X_train, y_train)

# Vorhersagen mit dem trainierten Modell
y_pred = svm_model.predict(X_test)

# Auswerten der Leistung des Modells
accuracy = accuracy_score(y_test, y_pred)
print(f"Genauigkeit des SVM-Modells: {accuracy}")

SVM Model Accuracy: 0.8297213622291022


In [28]:
# Speichern des trainierten Modells als Pickle-String.
saved_model = pickle.dumps(svm_model)

# Speichern des Modells auf 'raid/svm_model.pkl'
with open('raid/svm_model.pkl', 'wb') as file:
    pickle.dump(svm_model, file)

### Hyperparameter Tunning des SVM-Modells

In [31]:
from sklearn.model_selection import GridSearchCV

# Definieren des Parametergitters
param_grid = {
    'C': [0.1, 1, 10],  # Regularisierungsparameter
    'gamma': [1, 0.01, 'scale'],  # Kernel-Koeffizient
    'kernel': ['rbf', 'linear']  # Art des SVM
}

# Initialisieren des SVM-Modells
svm_model = SVC()

# Initialisieren des GridSearch-Modells
grid_search = GridSearchCV(svm_model, param_grid, cv=5, verbose=2, n_jobs=-1)

# Anpassen von GridSearch an die Trainingsdaten
grid_search.fit(X_train, y_train)

# Erhalten der besten Parameter
best_params = grid_search.best_params_
print(f"Beste Parameter: {best_params}")

# Trainieren des SVM-Modells mit den besten Parametern
svm_model = SVC(**best_params)
svm_model.fit(X_train, y_train)

# Vorhersagen mit dem trainierten Modell
y_pred = svm_model.predict(X_test)

# Auswerten der Leistung des Modells
accuracy = accuracy_score(y_test, y_pred)
print(f"Genauigkeit des SVM-Modells mit besten Parametern: {accuracy}")

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END ....................C=1, gamma=scale, kernel=linear; total time=38.9min
[CV] END .....................C=1, gamma=0.01, kernel=linear; total time=39.0min
[CV] END ...................C=0.1, gamma=0.01, kernel=linear; total time=39.1min
[CV] END .....................C=1, gamma=0.01, kernel=linear; total time=39.2min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=39.2min
[CV] END .......................C=10, gamma=1, kernel=linear; total time=39.3min
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=39.4min
[CV] END ........................C=1, gamma=1, kernel=linear; total time=39.4min
[CV] END ...................C=0.1, gamma=0.01, kernel=linear; total time=39.4min
[CV] END ........................C=1, gamma=1, kernel=linear; total time=39.4min
[CV] END ....................C=1, gamma=scale, kernel=linear; total time=39.5min
[CV] END ........................C=1, gamma=1, k

### Training mit Random Forest

In [56]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X_features, binary_targets, test_size=0.2, random_state=42)

# Initialize the Random Forest model with the best parameters
rf_model = RandomForestClassifier(n_estimators=100)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# Predict using the trained model
y_pred = rf_model.predict(X_test)

# Evaluate the model's performance
accuracy = rf_model.score(X_test, y_test)
print(f"Random Forest Model Accuracy: {accuracy}")

Random Forest Model Accuracy: 0.7647058823529411


### Hyperparameter Tunning des Random-Forest-Modells

In [175]:
param_grid = {
    'max_depth': [10, 20, 30, 40],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 'log2', None]
}

grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, cv = 3, n_jobs = -1)

grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

In [176]:
best_params

{'max_depth': 20,
 'max_features': None,
 'min_samples_leaf': 4,
 'min_samples_split': 2}

Die SVM-Genauigkeit bleibt auch nach dem Tuning des Random-Forest-Modells höher.

### Training mit Decision Tree

In [57]:
from sklearn.tree import DecisionTreeClassifier

# Aufteilen der Daten in Trainings- und Testsets
X_train, X_test, y_train, y_test = train_test_split(X_features, binary_targets, test_size=0.1, random_state=42)

# Initialisieren des DecisionTree-Modells mit den besten Parametern
dt_model = DecisionTreeClassifier(max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=2)

# Trainieren des Modells mit den Trainingsdaten
dt_model.fit(X_train, y_train)

# Vorhersagen mit dem trainierten Modell
y_pred = dt_model.predict(X_test)

# Auswerten der Leistung des Modells
accuracy = dt_model.score(X_test, y_test)
print(f"Genauigkeit des Decision-Tree-Modells: {accuracy}")

Decision Tree Model Accuracy: 0.6563467492260062


### Training mit KNN (K-nearest neighbors)

In [69]:
from sklearn.neighbors import KNeighborsClassifier

# Aufteilen der Daten in Trainings- und Testsets
X_train, X_test, y_train, y_test = train_test_split(X_features, binary_targets, test_size=0.2, random_state=42)

# Initialisieren des KNN-Modells
knn_model = KNeighborsClassifier(n_neighbors=100)  

# Trainieren des Modells mit den Trainingsdaten
knn_model.fit(X_train, y_train)

# Vorhersagen mit dem trainierten Modell
y_pred = knn_model.predict(X_test)

# Auswerten der Leistung des Modells
accuracy = accuracy_score(y_test, y_pred)
print(f"Genauigkeit des KNN-Modells: {accuracy}")

KNN Model Accuracy: 0.5201238390092879


## Inferenz

Das SVM hat sich als der leistungsfähigste erwiesen, weshalb er für die Inferenz verwendet wird.

In [12]:
def harvest_classif_model(svm_model, image_path):
    # Extrahieren der Merkmale mit VGG16 aus dem Bild
    VGG16_img_features = VGG16_feature_extratctor(image_path).flatten() 
    VGG16_img_features = VGG16_img_features.reshape(1, -1) 
    # Vorhersagen mit dem trainierten Modell
    y_pred = svm_model.predict(VGG16_img_features)

    return y_pred

In [13]:
# Laden des trainierten SVM-Modells
with open('raid/svm_model.pkl', 'rb') as file:
    svm_model = pickle.load(file)

In [20]:
image_path = 'raid/GrowliFlowerO/Field1/2020_10_29/2020_10_29_Ref_Plot1_A3.jpg'     # !!!!!!!!! Den Pfad entsprechend bearbeiten !!!!!!!!!
#image_path = 'raid/GrowliFlowerO/Field1/2020_10_27/2020_10_27_Ref_Plot1_A3.jpg'   # Drei Tage vorher

predicted_label = harvest_classif_model(svm_model, image_path)
if bool(predicted_label.item()):
    print('Die Pflanze ist geerntet')
else:
    print('Die Pflanze ist nicht geerntet')

1/1 [==============================] - 0s 101ms/step
Die Pflanze ist geerntet
